In [ ]:
import pandas as pd
import pyprind
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.io as pio
pio.renderers.default = 'vscode'

## RNN


#### Creating datasets

In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(cdf, test_size=0.2, random_state=10)


train_data, valid_data = train_test_split(train_data, test_size=0.1, random_state=10)


print(len(train_data))


print(cdf.head())

167093
                                                text  class
0  ['wife', 'threaten', 'suicide', 'recently', 'l...      1
1  ['weird', 'get', 'affect', 'come', 'someone', ...      0
2  ['finally', 'almost', 'never', 'hear', 'bad', ...      0
3    ['need', 'help', 'just', 'help', 'cry', 'hard']      1
4  ['lose', 'hello', 'name', 'struggle', 'year', ...      1


In [ ]:
from collections import Counter, OrderedDict
# Convert train_dataset to DataFrame
def tokenizer(text):
    tokenized = text.split()
    return tokenized
token_counts = Counter()
for text in train_data['text']:
    tokens = tokenizer(text)
    token_counts.update(tokens)

print('Vocab Size:', len(token_counts))

print(train_data.head())
    

Vocab Size: 37055
                                                     text  class
135094         ['hey', 'girl', 'tree', 'hear', 'whisper']      0
10717   ['girl', 'gon', 'need', 'help', 'mean', 'steal...      0
178488  ['want', 'make', 'only', 'fan', 'next', 'year'...      0
52509   ['even', 'brush', 'tooth', 'get', 'bed', 'with...      1
172712  ['dump', 'well', 'pretty', 'upset', 'relations...      0


In [ ]:
## Encoding Tokens to Integers
from torchtext.vocab import vocab

sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse= True)

ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocab = vocab(ordered_dict)
vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)
vocab.set_default_index(0)
print(train_data.head())

                                                     text  class
135094         ['hey', 'girl', 'tree', 'hear', 'whisper']      0
10717   ['girl', 'gon', 'need', 'help', 'mean', 'steal...      0
178488  ['want', 'make', 'only', 'fan', 'next', 'year'...      0
52509   ['even', 'brush', 'tooth', 'get', 'bed', 'with...      1
172712  ['dump', 'well', 'pretty', 'upset', 'relations...      0


# THIS IS THE PROBLEM!!!

In [ ]:
print([vocab[token] for token in ['this', 'dump', 'a', 'test']])

[0, 0, 0, 0]


In [ ]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]

label_pipeline = lambda x: 1. if x== 1 else 0.

    
print(train_data.head())

                                                     text  class
135094         ['hey', 'girl', 'tree', 'hear', 'whisper']      0
10717   ['girl', 'gon', 'need', 'help', 'mean', 'steal...      0
178488  ['want', 'make', 'only', 'fan', 'next', 'year'...      0
52509   ['even', 'brush', 'tooth', 'get', 'bed', 'with...      1
172712  ['dump', 'well', 'pretty', 'upset', 'relations...      0


# TASKS FOR Today:
### Build 3 RNN Models using these three types of recurrent layers
#### RNN
#### LSTM
#### GRU


In [ ]:
import torch
def collate_batch(batch):
    label_list, text_list, length = [], [], []
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text),
                                      dtype=torch.int64)
        text_list.append(processed_text)
        length.append(processed_text.size(0))
    label_list = torch.tensor(label_list)
    length = torch.tensor(length)
    padded_text_list = nn.utils.rnn.pad_sequence(
        text_list, batch_first=True)
    return padded_text_list, label_list, length

from torch.utils.data import DataLoader
dataloader = DataLoader(train_data, batch_size=4, shuffle=False, collate_fn=collate_batch)

In [ ]:
text_batch, label_batch, length_batch = next(iter(dataloader))
print(text_batch)

KeyError: 0

In [ ]:
batch_size = 32
train_dl = DataLoader(train_data, batch_size=batch_size,shuffle=True,collate_fn=collate_batch)
valid_dl = DataLoader(valid_data, batch_size=batch_size,shuffle=False,collate_fn=collate_batch)
test_dl = DataLoader(test_data, batch_size=batch_size,shuffle=False,collate_fn=collate_batch)

print(len(train_dl))
print(len(train_data))

5222
167093


In [ ]:
embedding = nn.Embedding(num_embeddings= len(token_counts), embedding_dim=100, padding_idx=0)

In [ ]:

import numpy as np
import torch

class SGD:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate
        
    def step(self, rnn_model, gradients):
        for param_name, gradient in gradients.items():
            setattr(rnn_model, param_name, getattr(rnn_model, param_name) - self.learning_rate * gradient)


class RNN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Random Initial Weights
        self.Wx = np.random.randn(hidden_size, input_size) * 0.01
        self.Wxy = np.random.randn(hidden_size, hidden_size) * 0.01
        self.Wyz = np.random.randn(output_size, hidden_size) * 0.01
        self.Bh = np.zeros((hidden_size, 1))
        self.Bo = np.zeros((output_size, 1))
        
        self.learning_rate = learning_rate
        self.optimizer = None

    def forward(self, inputs, h_prev):
        self.a = np.dot(self.Wx, inputs) + np.dot(self.Wxy, h_prev) + self.Bh
        self.h = np.tanh(self.a)
        self.y = np.dot(self.Wyz, self.h) + self.Bo
        return self.y, self.h

    def backward(self, d_y, h, h_prev, inputs):
        d_Wyz = np.dot(d_y, self.h.T)
        d_Bo = d_y
        d_h = np.dot(self.Wyz.T, d_y)
        d_a = np.dot(1 - self.h ** 2, d_h)
        d_Bh = d_a
        d_Wxy = np.dot(d_a, h_prev.T)
        d_Wx = np.dot(d_a, inputs.T)
        d_h_prev = np.dot(self.Wxy.T, d_a)
        
        gradients = {
            'd_Wx': d_Wx,
            'd_Wxy': d_Wxy,
            'd_Wyz': d_Wyz,
            'd_Bh': d_Bh,
            'd_Bo': d_Bo,
            'd_h_prev': d_h_prev
        }
        return gradients
    
    def train_step(self, inputs, targets, h_prev):
        # Forward pass
        y_pred, hidden_state = self.forward(inputs, h_prev)

        # Calculate loss
        loss = np.mean((y_pred - targets)**2)

        # Backward pass
        gradients = self.backward(2 * (y_pred - targets), hidden_state, h_prev, inputs)

        # Update weights
        self.optimizer.step(self, gradients)

        return loss

# Example usage:
input_size = 10
hidden_size = 64
output_size = 2
learning_rate = 0.01

rnn_model = RNN(input_size, hidden_size, output_size, learning_rate)




In [ ]:
input_size = 10
hidden_size = 64
output_size = 1
learning_rate = 0.01

rnn_model = RNN(input_size, hidden_size, output_size, learning_rate)
rnn_model.optimizer = SGD(learning_rate)

def train_model(dataloader):  # Renamed the function to avoid conflict
    total_acc, total_loss = 0.0, 0.0
    for text_batch, label_batch, lengths in dataloader:
        h_prev = np.zeros((hidden_size, text_batch.shape[0]))  # Initialize hidden state
        
        loss = rnn_model.train_step(text_batch, label_batch, h_prev)
        
        total_loss += loss * text_batch.shape[0]
        
        pred = rnn_model.forward(text_batch, h_prev)[0]
        total_acc += ((pred >= 0.5).astype(float) == label_batch).astype(float).sum().item()
        
    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)

In [ ]:
rnn_model = RNN(input_size = 100, hidden_size = 64, output_size = 2)
learning_rate = 0.01
num_epochs = 10




KeyError: 10293

In [ ]:
def evaluate(dataloader):
    total_acc, total_loss = 0, 0
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            h_prev = np.zeros((hidden_size, text_batch.shape[0]))  # Initialize hidden state
            
            loss = rnn_model.train_step(text_batch, label_batch, h_prev)
            
            total_loss += loss * text_batch.shape[0]
            
            pred = rnn_model.forward(text_batch, h_prev)[0]
            total_acc += ((pred >= 0.5).astype(float) == label_batch).astype(float).sum().item()
        
    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)

In [ ]:
torch.manual_seed(1)
for epoch in range(num_epochs):
    acc_train, loss_train = train_model(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f}'
          f' val_accuracy: {acc_valid:.4f}')

KeyError: 129376